In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras import layers
from keras.optimizers import RMSprop
import tensorflow as tf
import keras_tuner as kt

In [2]:
df = pd.read_csv("2nddataset.csv")
df

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,15,1,1,1,1,1,6,...,0,6,8,5,12.666667,0,15.5,2.8,-4.06,Graduate
4420,1,1,2,15,1,1,19,1,1,10,...,0,6,6,2,11.000000,0,11.1,0.6,2.02,Dropout
4421,1,1,1,12,1,1,1,22,27,10,...,0,8,9,1,13.500000,0,13.9,-0.3,0.79,Dropout
4422,1,1,1,9,1,1,1,22,27,8,...,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,Graduate


In [3]:
target_df = pd.get_dummies(df['Target'])
df = df.drop(columns = 'Target')
df2 = pd.concat([df, target_df], axis = 1)

In [4]:
def binary(cell):
    if cell == False:
        return 0
    else:
        return 1
col_list = target_df.columns.tolist()

for col in col_list:
    target_df[col] = target_df[col].apply(binary)
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Dropout   4424 non-null   int64
 1   Enrolled  4424 non-null   int64
 2   Graduate  4424 non-null   int64
dtypes: int64(3)
memory usage: 103.8 KB


In [6]:

# target_names = pd.factorize(df['Target'])[1]
# df['Target'] = pd.factorize(df['Target'])[0]

categorical_df = df.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18]].copy()
numerical_df = df.iloc[:,[2,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]].copy()
# y = df[['Target']].copy()

def categorize(cell):
    return f'cat_{cell}'
col_to_adjust = ['Marital status',
                 'Application mode',
                 'Course',
                 'Previous qualification',
                 'Nacionality',
                 "Mother's qualification",
                 "Father's qualification",
                 "Mother's occupation",
                 "Father's occupation"]
for col in col_to_adjust:
    categorical_df[col] = categorical_df[col].apply(categorize)
categorical_df.head()


to_drop = ["Application mode",
           "Nacionality",
           "Mother's qualification",
           "Father's qualification"]
cat_df_2 = pd.get_dummies(categorical_df.drop(columns = to_drop, axis = 1))

from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
pca_cat_df = pd.DataFrame(pca.fit_transform(cat_df_2), columns = ['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6',
                                                                  'PCA7','PCA8','PCA9','PCA10'])


In [7]:
others = ['Daytime/evening attendance',
          'Displaced',
          'Educational special needs',
          'Debtor',
          'Tuition fees up to date',
          'Gender',
          'Scholarship holder',
          'International']
parsed_cat_df = pd.concat([pca_cat_df, categorical_df[others]], axis = 1)
parsed_cat_df

scaled_num_df = pd.DataFrame(StandardScaler().fit_transform(numerical_df), columns = numerical_df.columns)
final_df = pd.concat([parsed_cat_df, scaled_num_df], axis = 1)
final_df.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,-0.162154,0.028006,-0.872105,-0.003570,-0.282523,-0.503001,-0.167394,-0.311028,0.552113,-0.044081,...,-0.199273,-0.282442,-2.838337,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
1,-0.230437,-0.508812,-0.888338,-0.126035,-0.399452,0.014727,-0.084807,-0.040341,-0.106143,0.024764,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.659562,-0.199441,0.876222,-1.105222,0.347199
2,0.442844,0.679989,-1.137452,-0.197999,-0.015396,0.026755,-0.077026,-0.164136,0.022110,0.339193,...,-0.199273,-0.282442,-0.105726,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
3,-0.720102,-0.190383,-0.002998,-0.299440,-0.633912,-0.210502,-0.347280,-0.193993,0.110501,-0.135233,...,-0.199273,-0.282442,-0.105726,0.490616,0.187165,0.416450,-0.199441,-0.813253,-1.466871,-1.375511
4,1.459905,0.612461,0.796145,-0.487428,-0.180104,-0.688034,0.328954,-0.473775,-0.069030,0.050491,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.531608,-0.199441,0.876222,-1.105222,0.347199


In [8]:
import numpy as np
X = final_df.copy()
y = target_df.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train = np.asarray(y_train).astype('int64').reshape((-1,1))
# y_test = np.asarray(y_test).astype('int64').reshape((-1,1))



In [12]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
    
    nn_model.add(tf.keras.layers.Dense(
        units = hp.Int('first_units', min_value = 1, max_value = 16, step = 2),
        activation = activation
    ))
    
    
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=1, max_value=12, step=2),
            activation=activation
        ))
    
    nn_model.add(tf.keras.layers.Dense(3, activation = 'sigmoid'))
    nn_model.compile(loss = "binary_crossentropy", optimizer = 'Adam', metrics = ["accuracy"])
    
    return nn_model

In [13]:
tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = 20,
    hyperband_iterations = 2,
    overwrite = True,
    
)

In [14]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))
#for i in range(0, len(tuner.get_best_models())):
    #print(tuner.get_best_models()[i])
best_model = tuner.get_best_models()

Trial 60 Complete [00h 00m 04s]
val_accuracy: 0.7459312677383423

Best val_accuracy So Far: 0.7613019943237305
Total elapsed time: 00h 02m 18s


In [16]:
for model in best_model:
    print(best_model)